In [ ]:
import numpy as np
import scipy
from scipy import stats
import time

from init import init_Y
from metrics import X_in_A, rho, eta, ESS, prop_alive_func
from stats import sampling_params, random_walk_on_params, pi_n, X_update, metropolis_X_theta

In [ ]:
# Step 0 : initialisation
alpha = 0.9
N = 1000
Nt = 500
Y = init_Y()
Xs = [[1]]*N
epsilon_final = 0.00045
epsilon_t = 1000
espilon_list = [epsilon_t]
random_walk_stds = (0.05, 0.05, 0.7)
Ws = 1/N * np.ones(N)
prop_alive = prop_alive_func(Ws)
thetas = np.array([sampling_params() for _ in range(N)])

for index, X in enumerate(Xs): # We do an updating so the Xs are not all the same at the beginning
    phi, tau, xi = thetas[index]
    X, phi, tau, xi= metropolis_X_theta(X, Y, phi, tau, xi,
                                        epsilon_t, random_walk_stds)
    thetas[index] = (phi, tau, xi)

In [ ]:
t = 0
while epsilon_t > epsilon_final:
    print(f'Step 1 : t={t}') # Step 1 : sampling

    new_weights = Ws
    iteration = 0
    while (prop_alive_func(new_weights) > alpha*prop_alive_func(Ws)) or (np.sum(new_weights) == 0):
        new_weights = Ws * np.array([int(X_in_A(Y, X, epsilon_t)) for X in Xs])

        amortization_factor = np.exp(-iteration/10)
        if np.sum(new_weights) == 0: # epsilon_t is too small
            epsilon_t *= 1 + 0.1 * amortization_factor
        else: # epsilon_t is too big
            epsilon_t *= 1 - 0.1 * amortization_factor
        print(prop_alive_func(new_weights), prop_alive_func(Ws), epsilon_t)
        iteration += 1



        time.sleep(0.5)
    Ws = new_weights
    if epsilon_t < epsilon_final:
        epsilon_t = epsilon_final
    espilon_list.append(epsilon_t)

    print('Step 2') # Step 2 : resampling
    if ESS(Ws) < Nt:
        indices = np.random.choice(range(N), size=N, replace=True, p=Ws)
        Xs = [Xs[i] for i in indices]
        thetas = [thetas[i] for i in indices]
        Ws = 1/N * np.ones(N)
        
    print('Step 3') # Step 3 : random walk
    for index, X in enumerate(Xs):
        if Ws[index] > 0:
            phi, tau, xi = thetas[index]
            X, phi, tau, xi= metropolis_X_theta(X, Y, phi, tau, xi,
                                                       epsilon_t, random_walk_stds)
            thetas[index] = (phi, tau, xi)
    t+=1
    print(f'espilon_t = {epsilon_t}, epsilon_final = {epsilon_final}')